In [2]:
#Barcha kerakli kutubxonalarni chaqirib olamiz

import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [3]:
stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

LR_model = LinearRegression()
DT_model = DecisionTreeRegressor()
RF_model = RandomForestRegressor()

encoder = OneHotEncoder()

In [4]:
#Datasetlarni yuklash
df_Train = pd.read_csv('train_data.csv', index_col=0)
df_Train.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
id,,,,,,,,,,,
1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [5]:
#Datasetdagi nan qiymatlarni tekshiramiz
df_Train.isnull().sum()

airline             0
flight              0
source_city         0
departure_time      0
stops               0
arrival_time        0
destination_city    0
class               0
duration            0
days_left           0
price               0
dtype: int64

In [6]:
df_Train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 1 to 20000
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   airline           20000 non-null  object 
 1   flight            20000 non-null  object 
 2   source_city       20000 non-null  object 
 3   departure_time    20000 non-null  object 
 4   stops             20000 non-null  object 
 5   arrival_time      20000 non-null  object 
 6   destination_city  20000 non-null  object 
 7   class             20000 non-null  object 
 8   duration          20000 non-null  float64
 9   days_left         20000 non-null  int64  
 10  price             20000 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 1.8+ MB


In [7]:
df_Train.describe()

,duration,days_left,price
count,20000.000000,20000.000000,20000.000000
mean,12.177627,25.924150,20960.281700
std,7.157944,13.624874,22775.459535
min,0.830000,1.000000,1105.000000
25%,6.830000,14.000000,4783.000000
50%,11.250000,26.000000,7425.000000
75%,16.080000,38.000000,42521.000000
max,38.580000,49.000000,114523.000000


In [8]:
df_Train['from_to'] = df_Train['source_city'] + '_' + df_Train['destination_city']

In [9]:
df_Train.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price,from_to
id,,,,,,,,,,,,
1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212,Bangalore_Mumbai
2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292,Hyderabad_Kolkata
3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553,Bangalore_Delhi
4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760,Kolkata_Hyderabad
5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712,Chennai_Mumbai


In [10]:
df_Train['from_to'].value_counts()

Mumbai_Delhi           1002
Delhi_Mumbai            981
Mumbai_Bangalore        920
Bangalore_Mumbai        909
Delhi_Bangalore         885
Bangalore_Delhi         884
Mumbai_Kolkata          824
Delhi_Kolkata           808
Kolkata_Mumbai          774
Delhi_Chennai           730
Kolkata_Delhi           711
Mumbai_Hyderabad        704
Bangalore_Kolkata       669
Kolkata_Bangalore       663
Hyderabad_Mumbai        659
Chennai_Delhi           652
Mumbai_Chennai          618
Delhi_Hyderabad         618
Chennai_Mumbai          595
Hyderabad_Delhi         576
Bangalore_Hyderabad     568
Hyderabad_Bangalore     539
Hyderabad_Kolkata       535
Kolkata_Hyderabad       492
Chennai_Kolkata         487
Bangalore_Chennai       456
Kolkata_Chennai         452
Hyderabad_Chennai       439
Chennai_Hyderabad       432
Chennai_Bangalore       418
Name: from_to, dtype: int64

In [11]:
df_Train.drop(['source_city', 'destination_city'], axis=1, inplace=True)

In [12]:
df_Train.head()

,airline,flight,departure_time,stops,arrival_time,class,duration,days_left,price,from_to
id,,,,,,,,,,
1,Vistara,UK-810,Early_Morning,one,Night,Economy,14.25,21,7212,Bangalore_Mumbai
2,SpiceJet,SG-5094,Evening,zero,Night,Economy,1.75,7,5292,Hyderabad_Kolkata
3,Vistara,UK-846,Morning,one,Evening,Business,9.58,5,60553,Bangalore_Delhi
4,Vistara,UK-706,Morning,one,Evening,Economy,6.75,28,5760,Kolkata_Hyderabad
5,Indigo,6E-5394,Early_Morning,zero,Morning,Economy,2.00,4,10712,Chennai_Mumbai


In [13]:
df_Train.corrwith(df_Train['price']).sort_values(ascending=False)

price        1.000000
duration     0.213158
days_left   -0.102545
dtype: float64

In [14]:
for train_index, test_index in stratified_split.split(df_Train,df_Train['class']):
    X_train_set = df_Train.iloc[train_index]
    X_test_set = df_Train.iloc[test_index]

In [15]:
X_train = X_train_set.drop(['price'], axis=1)
y = X_train_set['price'].copy()

In [16]:
num_pipeline = Pipeline([
          ('std_scaler', StandardScaler())
])

In [17]:
X_train

,airline,flight,departure_time,stops,arrival_time,class,duration,days_left,from_to
id,,,,,,,,,
6673,Vistara,UK-995,Morning,zero,Afternoon,Economy,2.25,36,Delhi_Mumbai
15137,Vistara,UK-877,Morning,one,Morning,Economy,22.08,18,Mumbai_Bangalore
14165,GO_FIRST,G8-2508,Night,one,Morning,Economy,9.92,47,Mumbai_Hyderabad
8070,Vistara,UK-720,Early_Morning,one,Evening,Economy,12.67,49,Kolkata_Hyderabad
10805,GO_FIRST,G8-424,Early_Morning,one,Evening,Economy,10.08,11,Hyderabad_Mumbai
...,...,...,...,...,...,...,...,...,...
14126,Vistara,UK-880,Afternoon,one,Night,Economy,9.25,32,Hyderabad_Bangalore
19260,Vistara,UK-811,Early_Morning,two_or_more,Night,Economy,17.17,8,Delhi_Chennai
947,Vistara,UK-707,Evening,zero,Evening,Business,2.08,21,Delhi_Kolkata


In [18]:
num_attribs = X_train.select_dtypes(include=['int64','float64','int32']).columns
cat_attribs = ['airline', 'departure_time', 'stops', 'arrival_time', 'class','from_to']

full_pipeline = ColumnTransformer([
              ('num', num_pipeline, num_attribs),
              ('cat', OneHotEncoder(), cat_attribs)
])

In [19]:
X_train = full_pipeline.fit_transform(X_train)

In [20]:
LR_model.fit(X_train, y)
X_test = X_test_set.drop('price', axis=1)
y_test = X_test_set['price'].copy()
X_test_prepared = full_pipeline.fit_transform(X_test)

y_predicted_lr = LR_model.predict(X_test_prepared)

mae = mean_absolute_error(y_test, y_predicted_lr)
mse = mean_squared_error(y_test, y_predicted_lr)

print("MAE=", np.around(mae))
print("RMSE=", np.around(np.sqrt(mse)))

MAE= 4571.0
RMSE= 6707.0


In [21]:
RF_model.fit(X_train,y)

y_predicted_rf = RF_model.predict(X_test_prepared)

mae = mean_absolute_error(y_test, y_predicted_rf)
mse = mean_squared_error(y_test, y_predicted_rf)

print("MAE=", np.around(mae))
print("RMSE=", np.around(np.sqrt(mse)))

MAE= 2113.0
RMSE= 4067.0


In [22]:
DT_model.fit(X_train,y)

y_predicted_dt = DT_model.predict(X_test_prepared)

mae = mean_absolute_error(y_test, y_predicted_dt)
mse = mean_squared_error(y_test, y_predicted_dt)

print("MAE=", np.around(mae))
print("RMSE=", np.around(np.sqrt(mse)))

MAE= 2469.0
RMSE= 5333.0


In [23]:
df = pd.read_csv('sample_solution.csv',index_col=0)

In [24]:
df.head()

,price
id,
1,0
2,0
3,0
4,0
5,0


In [25]:
df_Test = pd.read_csv('test_data.csv',index_col=0)
df_Test.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
id,,,,,,,,,,
1,Air_India,AI-765,Kolkata,Evening,one,Night,Delhi,Business,28.25,2
2,Vistara,UK-747,Delhi,Early_Morning,one,Night,Mumbai,Business,13.83,34
3,Air_India,AI-570,Mumbai,Early_Morning,zero,Early_Morning,Chennai,Business,2.00,30
4,AirAsia,I5-974,Hyderabad,Night,one,Late_Night,Delhi,Economy,5.17,26
5,Air_India,AI-770,Kolkata,Night,one,Afternoon,Mumbai,Economy,16.33,35


In [26]:
df_Test['from_to'] = df_Test['source_city'] + '_' + df_Test['destination_city']
df_Test.drop(['source_city', 'destination_city'], axis=1, inplace=True)

In [27]:
df_Test_prepared = full_pipeline.transform(df_Test)

predicted = RF_model.predict(df_Test_prepared)

In [28]:
predicted.shape

(5000,)

In [29]:
df.shape

(5000, 1)

In [30]:
df.price = predicted

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 1 to 5000
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   5000 non-null   float64
dtypes: float64(1)
memory usage: 78.1 KB


In [32]:
df.to_csv('submission.csv')

In [33]:
pd.read_csv('submission.csv')

,id,price
0,1,57850.48
1,2,57082.07
2,3,22969.98
3,4,2286.95
4,5,6031.29
...,...,...
4995,4996,60960.02
4996,4997,4530.97
4997,4998,50280.89
4998,4999,47837.80
